# Probabilistic Programming 4: Particle filter
## Monte Carlo sampling


### Preliminaries

- Goal 
  - Learn to apply Turing to a dynamical system.
- Materials        
  - Mandatory
    - These lecture notes.    
  - Optional
      - [Probabilistic Programming notebook](https://github.com/bertdv/BMLIP/tree/master/lessons/notebooks/probprog/Probabilistic-Programming.ipynb)
      - David Blei (https://doi.org/10.1146/annurev-statistics-022513-115657)

In [ ]:
# Package managing
using Pkg
Pkg.activate("workspace")
Pkg.instantiate();

using Logging; disable_logging(LogLevel(0))
using Distributions
using StatsPlots
using Turing
using MCMCChains
Turing.setadbackend(:forward_diff)
include("../scripts/pp-4.jl");

### Data generation

In this notebook, we will be considering continuous state-space models. We will focus on the simplest form: the Gaussian dynamical system:

$$\begin{align}
x_k =&\ A_kx_{k-1} + q_{k-1} \\
y_k =&\ H_kx_k + r_k \, ,
\end{align}$$

where $x_k \in \mathbb{R}^{N}$ is the state, $y_k \in \mathbb{R}^{M}$ is the measurement. Process noise and measurement noise are assumed to be zero mean Gaussian with covariance matrices $Q_k$ and $R_k$ respectively. In Bayesian notation this takes the form:

$$\begin{align}
p(x_k \mid x_{k-1}) =&\ \mathcal{N}(x_k \mid A_k x_{k-1}, Q_{k-1})\\
p(y_k \mid x_k) =&\ \mathcal{N}(y_k \mid H_k x_k, R_k) \, .
\end{align}$$

We can sample from these distributions to generate data. We'll consider a $2$-dimensional state (i.e. $N=2$) and a $2$-dimensional observation (i.e. $M=2$). First, we set the initial state $x_0$ to some value and generate $x_1$ through the state transition. Then, we generate the observation $y_1$ from the generated state $x_1$.

In [ ]:
# Dimensionalities
N = 2
M = 2

# Length of time-series
T = 50

# Lenght of time step
Δt = 1.0

# Transition matrix of latent variables
transition = [1.0 Δt;
              0.0 1.0]

# Emission matrix for observed variables
emission = [1.0 0.;0. 1.0]

# Process noise (latent variables)
process_noise = [0.2*Δt 0.;0. 0.1*Δt]

# Measurement noise (observations)
measurement_noise = [0.1 0.;0. 0.1]

# Preallocate data arrays
states = zeros(2,T)
observations = zeros(2,T)

# Initial state
state_0 = [0., 0.]

# Keep previous state in memory
state_tmin = state_0

# Generate data for entire time-series
for t = 1:T
    
    # Transition from previous state
    states[:,t] = rand(MvNormal(transition * state_tmin, process_noise), 1)[:,1]
    
    # Emission of current state
    observations[:,t] = rand(MvNormal(emission * states[:,t], measurement_noise), 1)[:,1]
    
    # Update previous state
    state_tmin = states[:,t]
    
end

# Visualization of states
plot(states[1,:], states[2,:], color="red", label="states", grid=false, xlabel="position x-coordinate", ylabel="position y-coordinate")
scatter!(observations[1,:], observations[2,:], color="blue", label="observations")

Since we don't have a time-axis anymore, it is hard to see how the process evolves. Animating the path resolves this.

In [ ]:
# Animation of cart's path

# Plot initial state
plot([states[1,1]], [states[2,1]], 
     color="red", 
     label="states", 
     grid=false, 
     xlabel="position x-coordinate", 
     ylabel="position y-coordinate", 
     xlims=[-60,60], 
     ylims=[-10,10])

# Plot initial observation
scatter!([observations[1,1]], [observations[2,1]], color="blue", label="observations")

anim = @animate for t = 2:T
    
    title!("t = "*string(t))
    
    # Plot new state
    plot!([states[1,t-1:t]], [states[2,t-1:t]], color="red", label="")
    
    # Plot new observation
    scatter!([observations[1,t]], [observations[2,t]], color="blue", label="")
    
end

gif(anim, "visualizations/example-GDS.gif", fps = 3);

## Model specification

We will employ a Particle Filter to estimate states and unknown parameters. Particle filters are the basic means of doing Markov Chain Monte Carlo sampling in continous state-space models. They are also known as _Sequential Importance Sampling_ or _Sequential Monte Carlo_ methods. Particle filters essentially sample from the state transition, taking the sample average as the expected new state, and sample from the observation emission, taking the sample average as the expected observation. It will update the expected step in the same way as the Kalman filter: by multiplying the prediction and corrections:

$$ \hat{p}(x_t \mid y_{1:T}) \approx \hat{p}(x_t \mid x_{t-1}) \hat{p}(y_t \mid x_t) \, ,$$

where $\hat{\cdot}$ indicates that these distributions are approximated. Particle filters can be applied to non-Gaussian dynamical sytems. In that case, the probabilities of the samples draw from an approximating distribution (e.g. Gaussian) are importance-weighted to resemble the probability distribution of the non-Gaussian dynamical system.

Again, we start simple and slowly improve the model.

### Model 1: estimate states

We will first assume we know the transition and emission matrices. Based on these, we purely want to estimate states from observations.

In [ ]:
# Turing model definition.
@model PF(y, transition, process_noise, emission, measurement_noise) = begin
    "This model implicitly assumes 2-dim states and 2-dim observations"
    
    # Time-series length
    T = size(y, 2)

    # State sequence.
    x = Vector{Vector}(undef, T)

    # Define initial state
    x_0 ~ MvNormal([0., 0.], [1. 0.;0. 1.])
    
    # Initialize previous state
    x_tmin = x_0

    # Loop over entire sequence
    for t = 1:T
        
        # State transition     
        x[t] ~ MvNormal(transition * x_tmin, process_noise)
        
        # Observation emission
        y[:,t] ~ MvNormal(emission * x[t], measurement_noise)
        
        # Update previous state
        x_tmin = x[t]
    end
end

# Call instance of the model
model1 = PF(observations, transition, process_noise, emission, measurement_noise);

We will use a "Sequential Monte Carlo" sampler, specifying its number of particles as a parameter.

In [ ]:
# Length of chain
len_chain = 200

# Define sampler
sampler1 = Gibbs(PG(100, :x))

# Call sampler
chain1 = sample(model1, sampler1, len_chain);

Let's look at the chain.

In [ ]:
chain1[:x]

And plot the inferred states.

In [ ]:
# Extract mean of the chain
x_hat = reshape(mean(chain1[:x].value.data, dims=[1]), (2,50))

# Extract standard error of the mean
x_sem = reshape(std(chain1[:x].value.data, dims=[1]), (2,50))
x_sem[1,:] ./= sqrt.(range(1, stop=T))
x_sem[2,:] ./= sqrt.(range(1, stop=T));

# Visualization
plot(states[1,:], states[2,:], color="red", label="states", grid=false)
plot!(x_hat[1,:], x_hat[2,:], ribbon=[x_sem, x_sem], fillalpha=0.2, color="green", label="inferred")
scatter!(observations[1,:], observations[2,:], color="blue", label="observations")

### Model 2: transition matrix estimation

At the moment, we assume we know how the states evolve over time. This is not an unreasonable assumption; in protein folding we know exactly _how_ the protein changes state, i.e. through the signaling of another molecule, and we know the concentration of signal molecules. However, in other applications we do not necessarily know the transition matrix. 

We can estimate the transition matrix by letting the coefficients be latent variables and posing a prior.